In [1]:
### How good is a batsman against 3 spinners playing in wankhede?


# Venue:Wankhede
# Phase: Middle
# Opposition: Mumbai Indians

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',-1)

In [3]:
def get_phase(over):
    if over<=6:
        return 'Powerplay'
    elif over<=15:
        return 'Middle'
    else:
        return 'Death'
    
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1
        
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [4]:
def byCustom(df, current_venue, current_phase, current_opposition):
    
    df = df[df.venue==current_venue]
    df = df[df.phase==current_phase]
    df = df[df.bowling_team==current_opposition]

    df.reset_index(inplace=True, drop= True)

    df['isdot'] = df['batsman_runs'].apply(lambda x:1 if x==0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x:1 if x==1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x:1 if x==2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x:1 if x==3 else 0)
    df['isFour'] = df['batsman_runs'].apply(lambda x:1 if x==4 else 0)
    df['isSix'] = df['batsman_runs'].apply(lambda x:1 if x==6 else 0)
#     df['ispowerplay']=df['over'].apply(lambda x:1 if x<=6 else 0)
#     df['ismiddle']=df['over'].apply(lambda x:1 if (x>6 and x<16) else 0)
#     df['isdeath']=df['over'].apply(lambda x:1 if (x>=16 and x<20) else 0)


    
    pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].sum()).head(3).index
    runs=(pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].sum()).reset_index()).rename(columns={'batsman_runs':'runs'})
    #(df.groupby(['batsman'])['match_id'].unique())
    #df.groupby(['batsman'])['match_id'].count()
    balls=pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    innings=pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    dismissals=pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})
    Sixes=(pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index()).rename(columns={'isSix':'Sixes'})
    fours=(pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index()).rename(columns={'isFour':'fours'})
    dots=(pd.DataFrame(df.groupby(['batsman'])['isdot'].sum()).reset_index()).rename(columns={'isdot':'dots'})
    
#     powerplay=df['over'].apply(lambda x:1 if x<=6 else 0)
#     middle=df['over'].apply(lambda x:1 if (x>6 and x<16) else 0)
#     death=df['over'].apply(lambda x:1 if (x>=16 and x<20) else 0)
    
    
    df=pd.merge(innings,runs,on="batsman").merge(balls,on="batsman").merge(dismissals,on="batsman").merge(Sixes,on="batsman").merge(fours,on="batsman").merge(dots,on="batsman")
    df['RPI']=df.apply(lambda x: (x['runs']/x['innings']),axis=1)
    df['Strike_rate']=df.apply(lambda x: (x['runs']/x['balls'])*100,axis=1)
    df['BPD']=df.apply(lambda x: balls_per_dismissal(x['balls'],x['dismissals']), axis=1)
    df['BPB']=df.apply(lambda x: balls_per_boundary(x['balls'], (x['Sixes'] + x['fours'])), axis = 1)
    df['dot_percentage'] = df.apply(lambda x: (x['dots']/x['balls']),axis=1)

    return df;

In [5]:
deliveries = pd.read_csv("deliveries.csv")
df= deliveries.copy()

In [6]:
matches= pd.read_csv("matches.csv")
mdf = matches.copy()

In [7]:
mdf.head(1)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [8]:
mdf=mdf.rename(columns={'id':'match_id'})

In [9]:
comb=pd.merge(df,mdf,on="match_id",how="left")

In [10]:
comb.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [11]:
comb['phase'] = comb['over'].apply(lambda x: get_phase(x))

In [12]:
comb.venue.unique()

array(['Rajiv Gandhi International Stadium, Uppal',
       'Maharashtra Cricket Association Stadium',
       'Saurashtra Cricket Association Stadium', 'Holkar Cricket Stadium',
       'M Chinnaswamy Stadium', 'Wankhede Stadium', 'Eden Gardens',
       'Feroz Shah Kotla',
       'Punjab Cricket Association IS Bindra Stadium, Mohali',
       'Green Park', 'Punjab Cricket Association Stadium, Mohali',
       'Sawai Mansingh Stadium', 'MA Chidambaram Stadium, Chepauk',
       'Dr DY Patil Sports Academy', 'Newlands', "St George's Park",
       'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed V

In [35]:
df= byCustom(comb, 'Wankhede Stadium', "Middle", 'Mumbai Indians')

In [65]:
### FILTER TO REMOVE OUTLIERS
df = df[df.innings>1]
df.head()


,batsman,innings,runs,balls,dismissals,Sixes,fours,dots,RPI,Strike_rate,BPD,BPB,dot_percentage,calc_SR,calc_RPI,calc_BPD,calc_dot_percentage,dev_best_sr,dev_best_rpi,dev_best_BPD,dev_best_dot_percentage,dev_best_sqrt,dev_worst_sr,dev_worst_rpi,dev_worst_BPD,dev_worst_dot_percentage,dev_worst_sqrt,score
3,AB de Villiers,5,134,79,4,4,17,21,26.8,169.620253,19.750000,3.761905,0.265823,5.783718,1.917453,0.386334,0.016505,2.077973,0.290442,26.883134,0.002566,29.254114,19.038577,3.192083,0.082535,0.000009,22.313204,0.432700
10,AM Rahane,4,98,68,3,4,7,18,24.5,144.117647,22.666667,6.181818,0.264706,4.175284,1.602461,0.508866,0.016366,9.302204,0.729177,25.627510,0.002580,35.661472,7.589407,2.165746,0.167954,0.000008,9.923115,0.217686
15,BB McCullum,3,84,59,2,4,7,20,28.0,142.372881,29.500000,5.363636,0.338983,4.074799,2.093010,0.861931,0.026840,9.925247,0.132038,22.177484,0.001626,32.236394,7.045857,3.850216,0.581996,0.000177,11.478246,0.262572
20,CH Gayle,3,54,46,2,4,2,20,18.0,117.391304,23.000000,7.666667,0.434783,2.770279,0.864968,0.523943,0.044154,19.846635,2.532589,25.475090,0.000529,47.854844,1.822184,0.538984,0.180539,0.000937,2.542645,0.050452
29,DJ Bravo,4,48,49,1,1,4,22,12.0,97.959184,49.000000,9.800000,0.448980,1.929043,0.384430,2.378048,0.047084,28.049650,4.292973,10.196402,0.000403,42.539428,0.258722,0.064322,5.193859,0.001125,5.518028,0.114821


In [66]:
#sattys scale

In [67]:
wt_sr, wt_rpi, wt_bpd, wt_dot_Percentage= 0.13, 0.27, 0.16, 0.45

In [68]:
df['calc_SR']=df['Strike_rate'].apply(lambda x: x*x)
df['calc_RPI']=df['RPI'].apply(lambda x: x*x)
df['calc_BPD']=df['BPD'].apply(lambda x: x*x)
df['calc_dot_percentage']=df['dot_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_Percentage= np.sqrt(df[['calc_SR','calc_RPI','calc_BPD','calc_dot_percentage']].sum())

df['calc_SR']=df['calc_SR'].apply(lambda x: x/sq_sr)
df['calc_RPI']=df['calc_RPI'].apply(lambda x: x/sq_rpi)
df['calc_BPD']=df['calc_BPD'].apply(lambda x: x/sq_bpd)
df['calc_dot_percentage']=df['calc_dot_percentage'].apply(lambda x: x/sq_dot_Percentage)

df['calc_SR']=df['calc_SR'].apply(lambda x: x*wt_sr)
df['calc_RPI']=df['calc_RPI'].apply(lambda x: x*wt_rpi)
df['calc_BPD']=df['calc_BPD'].apply(lambda x: x*wt_bpd)
df['calc_dot_percentage']=df['calc_dot_percentage'].apply(lambda x: x*wt_dot_Percentage)



best_sr, worst_sr = max(df['calc_SR']),min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']),min(df['calc_RPI'])
best_BPD, worst_BPD = max(df['calc_BPD']),min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = max(df['calc_dot_percentage']),min(df['calc_dot_percentage'])

In [69]:
df['dev_best_sr']=df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr))
df['dev_best_rpi']=df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi))
df['dev_best_BPD']=df['calc_BPD'].apply(lambda x: (x-best_BPD)*(x-best_BPD))
df['dev_best_dot_percentage']=df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df['dev_best_sqrt']=df.apply(lambda x: (x['dev_best_sr']+ x['dev_best_rpi']+ x['dev_best_BPD']+ x['dev_best_dot_percentage']),axis=1)


df['dev_worst_sr']=df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr))
df['dev_worst_rpi']=df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi))
df['dev_worst_BPD']=df['calc_BPD'].apply(lambda x: (x-worst_BPD)*(x-worst_BPD))
df['dev_worst_dot_percentage']=df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df['dev_worst_sqrt']=df.apply(lambda x: (x['dev_worst_sr']+ x['dev_worst_rpi']+ x['dev_worst_BPD']+ x['dev_worst_dot_percentage']),axis=1)
#df.head(2)

In [70]:
df['score']=df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt']+x['dev_best_sqrt']),axis=1)
#df['batsman'].sort_values(by='score', ascending = False)

In [71]:
df.head()

,batsman,innings,runs,balls,dismissals,Sixes,fours,dots,RPI,Strike_rate,BPD,BPB,dot_percentage,calc_SR,calc_RPI,calc_BPD,calc_dot_percentage,dev_best_sr,dev_best_rpi,dev_best_BPD,dev_best_dot_percentage,dev_best_sqrt,dev_worst_sr,dev_worst_rpi,dev_worst_BPD,dev_worst_dot_percentage,dev_worst_sqrt,score
3,AB de Villiers,5,134,79,4,4,17,21,26.8,169.620253,19.750000,3.761905,0.265823,5.783718,1.917453,0.386334,0.016505,2.077973,0.290442,26.883134,0.002566,29.254114,19.038577,3.192083,0.082535,0.000009,22.313204,0.432700
10,AM Rahane,4,98,68,3,4,7,18,24.5,144.117647,22.666667,6.181818,0.264706,4.175284,1.602461,0.508866,0.016366,9.302204,0.729177,25.627510,0.002580,35.661472,7.589407,2.165746,0.167954,0.000008,9.923115,0.217686
15,BB McCullum,3,84,59,2,4,7,20,28.0,142.372881,29.500000,5.363636,0.338983,4.074799,2.093010,0.861931,0.026840,9.925247,0.132038,22.177484,0.001626,32.236394,7.045857,3.850216,0.581996,0.000177,11.478246,0.262572
20,CH Gayle,3,54,46,2,4,2,20,18.0,117.391304,23.000000,7.666667,0.434783,2.770279,0.864968,0.523943,0.044154,19.846635,2.532589,25.475090,0.000529,47.854844,1.822184,0.538984,0.180539,0.000937,2.542645,0.050452
29,DJ Bravo,4,48,49,1,1,4,22,12.0,97.959184,49.000000,9.800000,0.448980,1.929043,0.384430,2.378048,0.047084,28.049650,4.292973,10.196402,0.000403,42.539428,0.258722,0.064322,5.193859,0.001125,5.518028,0.114821


In [75]:
df[['batsman','score']].sort_values(by='score', ascending = False).reset_index()

,index,batsman,score
0,69,MK Tiwary,0.674576
1,40,GJ Maxwell,0.580473
2,103,SE Marsh,0.524171
3,3,AB de Villiers,0.432700
4,119,WP Saha,0.402412
5,15,BB McCullum,0.262572
6,10,AM Rahane,0.217686
7,99,S Dhawan,0.203939
8,122,Yuvraj Singh,0.168196
9,112,Shakib Al Hasan,0.139803
